# T81-558: Applications of Deep Neural Networks
**Module 10: Time Series in Keras**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 10 Material

* Part 10.1: Time Series Data Encoding for Deep Learning [[Video]]() [[Notebook]](t81_558_class_10_1_timeseries.ipynb)
* Part 10.2: Programming LSTM with Keras and TensorFlow [[Video]]() [[Notebook]](t81_558_class_10_2_lstm.ipynb)
* **Part 10.3: Text Generation with Keras and TensorFlow** [[Video]]() [[Notebook]](t81_558_class_10_3_text_generation.ipynb)
* Part 10.4: Image Captioning with Keras and TensorFlow [[Video]]() [[Notebook]](t81_558_class_10_4_captioning.ipynb)
* Part 10.5: Temporal CNN in Keras and TensorFlow [[Video]]() [[Notebook]](t81_558_class_10_5_temporal_cnn.ipynb)

# Part 10.3: Text Generation with LSTM

Recurrent neural networks are also known for their ability to generate text.  This can allow the output of the neural network to be free-form text.  In this part we will see how an LSTM can be trained on a textual document, such as classic literature, and learn to output new text that appears to be of the same form as the training material.  If you train your LSTM on [Shakespeare](https://en.wikipedia.org/wiki/William_Shakespeare), then it will learn to crank out new prose that is similar to what Shakespeare had written. 

Don't get your hopes up.  Your not going to each your deep neural network to write the next [Pulitzer Prize for Fiction](https://en.wikipedia.org/wiki/Pulitzer_Prize_for_Fiction).  The prose generated by your neural network will be nonsensical.  However, it will usually be nearly grammatically and of a similar style as the source training documents. 

A neural network generating nonsensical text based on literature may not seem terribly useful at first glance.  However, the reason that this technology gets so much interest is that it forms the foundation for many more advanced technologies.  The fact that the LSTM will typically learn human grammar from the source document opens a wide range of possibilities. Similar technology can be used to complete sentences when a user is entering text.  Simply the ability to output free-form text becomes the foundation of many other technologies.  In the next part, we will make use of this technique to create a neural network that can write captions for images to describe what is going on in the image. 

### Additional Information

The following are some of the articles that I found useful putting this section together.

* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* [Keras LSTM Generation Example](https://keras.io/examples/lstm_text_generation/)

### Character-Level Text Generation

There are a number of different approaches to teaching a neural network to output free-form text.  The most basic question is if you wish the neural network to learn at the word or character level.  In many ways, lerning at the character level is the more interesting of the two.  The LSTM is learning construct its own words without even being shown what a word is.  We will begin with character-level text generation.  In the next module, we will see how we can use nearly the same technique to operate at the word level.  The automatic captioning that will be implemented in the next module is at the word level.

We begin by importing the needed Python packages and defining the sequence length, named **maxlen**.  Time-series neural networks always accept their input as a fixed length array.  Not all of the sequence might be used, it is common to fill extra elements with zeros.  The text will be divided into sequences of this length and the neural network will be trained to predict what comes after this sequence.

In [1]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re

Using TensorFlow backend.


For this simple example we will train the neural network on the classic children's book [Treasure Island](https://en.wikipedia.org/wiki/Treasure_Island).  We begin by loading this text into a Python string and displaying the first 1,000 characters.

In [2]:
r = requests.get("https://data.heatonresearch.com/data/t81-558/text/treasure_island.txt")
raw_text = r.text
print(raw_text[0:1000])


ï»¿The Project Gutenberg EBook of Treasure Island, by Robert Louis Stevenson

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: Treasure Island

Author: Robert Louis Stevenson

Illustrator: Milo Winter

Release Date: January 12, 2009 [EBook #27780]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK TREASURE ISLAND ***




Produced by Juliet Sutherland, Stephen Blundell and the
Online Distributed Proofreading Team at http://www.pgdp.net









 THE ILLUSTRATED CHILDREN'S LIBRARY


         _Treasure Island_

       Robert Louis Stevenson

          _Illustrated by_
            Milo Winter


           [Illustration]


           GRAMERCY BOOKS
              NEW YORK




 Foreword copyright Â© 1986 by Random House V


We will extract all unique characters from the text and sort them.  This allows us to assign a unique ID to each character.  Because the characters are sorted, these IDs should remain the same.  If new characters were added to the original text, then the IDs would change.  We build up two dictionaries.  The first **char2idx** is used to convert a character into its ID.  The second **idx2char** converts an ID back into its character.

In [3]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]',r'', processed_text) 

In [4]:
print('corpus length:', len(processed_text))

chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 397400
total chars: 60


We are now ready to build the actual sequences.  Just like previous neural networks there will be an $x$ and $y$.  However, for the LSTM, $x$ and $y$ will both be sequences.  The $x$ input will specify the sequences where $y$ are the expected output.  The following code generates all possible sequences.

In [5]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 132454


In [6]:
sentences

['the project gutenberg ebook of treasure ',
 ' project gutenberg ebook of treasure isl',
 'oject gutenberg ebook of treasure island',
 'ct gutenberg ebook of treasure island, b',
 'gutenberg ebook of treasure island, by r',
 'enberg ebook of treasure island, by robe',
 'erg ebook of treasure island, by robert ',
 ' ebook of treasure island, by robert lou',
 'ook of treasure island, by robert louis ',
 ' of treasure island, by robert louis ste',
 ' treasure island, by robert louis steven',
 'easure island, by robert louis stevenson',
 'ure island, by robert louis stevenson\r\n\r',
 ' island, by robert louis stevenson\r\n\r\nth',
 'land, by robert louis stevenson\r\n\r\nthis ',
 'd, by robert louis stevenson\r\n\r\nthis ebo',
 'by robert louis stevenson\r\n\r\nthis ebook ',
 'robert louis stevenson\r\n\r\nthis ebook is ',
 'ert louis stevenson\r\n\r\nthis ebook is for',
 ' louis stevenson\r\n\r\nthis ebook is for th',
 'uis stevenson\r\n\r\nthis ebook is for the u',
 ' stevenson\r\n\r\n

In [7]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [8]:
x.shape

(132454, 40, 60)

In [9]:
y.shape

(132454, 60)

The dummy variables for $y$ are shown below.

In [10]:
y[0:10]

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, Fal

Next, we create the neural network.  The primary feature of this neural network is the LSTM layer.  This allows the sequences to be processed.  

In [11]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               96768     
_________________________________________________________________
dense_1 (Dense)              (None, 60)                7740      
Total params: 104,508
Trainable params: 104,508
Non-trainable params: 0
_________________________________________________________________


The LSTM will produce new text character by character.  We will need to sample the correct letter from the LSTM predictions each time.  The **sample** function accepts the following two parameters:

* **preds** - The output neurons.
* **temperature** - 1.0 is the most conservative, 0.0 is the most confident (willing to make spelling and other errors).

The sample function below is essentially performing a [softmax]() on the neural network predictions.  This causes each output neuron to become a probability of its particular letter.  

In [13]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [14]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


We are now ready to train.  It can take up to an hour to train this network, depending on how fast your computer is.  If you have a GPU available, please make sure to use it.

In [15]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
132454/132454 [==============================] - 59s 443us/step - loss: 2.0904
****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "t
information can be found at the found"
t
information can be found at the found the captain the sound a gond a a cood the cooked the cooked and the spear the stor and he had the cood and the ston the cood and the cooked the captain and he had the doctor the cood a beck of the captain the captain the ston the cood and to the stare we was a cood all the cood and a look of the coost a cood the captain and he captain and a look of the cood and a seat and he was a gond a cook and
----- temperature: 0.5
----- Generating with seed: "t
information can be found at the found"
t
information can be found at the found of the cooked as ever comed had were the stor the coon of the cound the wors to the cood the for the fard cook mur had
with a cond

the loophore to the beach, there went the sea starm and the atull and retern was the rost and so hear sking some that soon the
----- temperature: 1.0
----- Generating with seed: "of you in this here house, and what i sa"
of you in this here house, and what i say; "there me. "all, now ither wenolled at that, now,
this toby; but no engless, for after being outsilly. "be set to toil honse, grow they have fulletward at the stoom bir sea fanix of theisr. but if him, sea side corry sald thats of it have acread began about
ore puts.

"her, direp what soy lo
sire in dishowed's that his stapped about was an
your maris.

"armine of fullowed of running.
n
----- temperature: 1.2
----- Generating with seed: "of you in this here house, and what i sa"
of you in this here house, and what i says, might deads only or fad on tims and though loopt youcaglishy; he are seen donatiot sall, stood sost cormint, tell another and wimllest and fro or.

"a wild came
-acca
 the dorthed cutver, spy---o' bluck teap w

132454/132454 [==============================] - 54s 409us/step - loss: 1.4123
****************************************************************************
----- Generating text after Epoch: 7
----- temperature: 0.2
----- Generating with seed: " or received a letter,
and he never spo"
 or received a letter,
and he never spot be says her beck to the story and the story and seen the stockade on the surf bother of the stoc

/Users/jheaton/miniconda3/envs/wustl/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


kade and the stranger and seen the captain. "you see you was a botton of the back to the stopped to the story i say many short and the man and the captain and the stockade of the stockade on the stockade on the seams of the stockade of the stockade of the stockade and the story was a start and the wor
----- temperature: 0.5
----- Generating with seed: " or received a letter,
and he never spo"
 or received a letter,
and he never spot be go the brow to his one miect, when i had enterve and a surficion had men a sently to be would seen a man of the came of the back and believe the blower the taily and saved, and the man project gutenberg-tm the stopped to the words and had fallen about the was of my house and still never think some might in the armsing to the afternoot, and he came upon the same such began to be gave a short t
----- temperature: 1.0
----- Generating with seed: " or received a letter,
and he never spo"
 or received a letter,
and he never spowding it."

andest brought i get

...

a n ouh  ao  we torse aoanou ee   ce her  ent aih e  at a  hn n hhutnn s th yeu eouhee aet  ueere e tx d n
u hse to,ed in an 
----- temperature: 1.0
----- Generating with seed: "d his ship, and
in short, had so agreea"
d his ship, and
ln short, had so agreearose i u eso eo teetureodn oielse caoro
tti len  m eu sur ,ine nytrus m llhhme talr  hedj    w  t nuahinuinathettl
 t oawistlhhu owi inw  w r iheta   siehl,ash thc s henyoo cely al  ylrianl uetau cnhereulersn m 
at dve ws ht totlrmlnatu  stmd asoonuehcle
ehteotede tsie whloe?ld ss tde uhosetht wate ho cens did
weui ma cyee   antes,heiuhrei ilh thto mthnet ran  as u  ihnioe rir thmtohe 
----- temperature: 1.2
----- Generating with seed: "d his ship, and
in short, had so agreea"
d his ship, and
in short, had so agreeasnoi,nw, o hthe si e  l thaly dna csheles
ron eoot wt u saax th,hei so  mo hhyoeut itit
e,atoheneor , ou  h nurlelu tht, is herote ,eclere  wu
ehyrr lel e disitenisdis iuir  iw t anhetttnuiscou asadn
,ant e nal itsyojl,sd

****************************************************************************
----- Generating text after Epoch: 46
----- temperature: 0.2
----- Generating with seed: "r
distribute copies of a project gutenb"
r
distribute copies of a project gutenb e 
    a    he  no theadu a   an thh  h th wh  een t th t   th  o  h ree  the  a  h h o eer   h  hh hen i n  e t ou he   h h a  th  he   the  ha      he  an  an i   hh  h  her th h d  an  h t a u  h t e h an  h t   he  an the   a h   a e     ans ac ald h  the  rh    h   h  mn o th th  a ne ther
 h  i    an  h  nh th 
 he   u an an   he we  anh  h le  cer  h  thh h th e eh t  h  het whe a  th t 
----- temperature: 0.5
----- Generating with seed: "r
distribute copies of a project gutenb"
r
distribute copies of a project gutenboeo  o w o  t ar h tth ota d h h m iee r he e  an rdhh h tee te  th  her  
e tr e  ue i wer  a 
d ho
s hh_  u    trtn s s ci h ae   h h tliner  hnn at aoh r  
 a hrrarer 
 t ao  h  thion thrynior e hton  warceo rhin hth  h

rhen tmeunire wns andsd ani coheu, m w tt  r tisasc,er eanrehaoasimd,noheol 
 ch ohraradsen  a
 in  saere  ods ine wmonecdl, eoh asledomnt es m
 l es wwit sl  san, t eous
ictdwoe mua o rnsre l
s th etasli unn t yeahr , dy sen
 y
oane ss sor
,hd,aoa s m m thet lsomnat or
h ailw im dht  s huesa heth sndt
 l nr c mlruusthw rei t n aa nto  imnou  as a iish whain  iwaamhltweaenst ler adtily
eoh
----- temperature: 1.2
----- Generating with seed: "e arrival of the _hispaniola_.

when t"
e arrival of the _hispaniola_.

uhen tande,olat ens oitsoeteh  
oseas
uialmmhat e s

youiet d mw,d ceu aaou te,lohhar anure areui isayotneuirm   e ilu er d,yhhtnute  as yllane 
iat  mancasrcw,enh dnai  c siet c e oionoi welihd so aic ninat eel,s
hie oaii iaie,t ah lte ailermrao ry rleeaimdsdoic  c eaihe,n tlu iis
nrdhtc tomhsore esi wow r,oier ila e wylahitonire
 wewe nm irmaeln mthi  imindne tea itntutecd  woulnmel ha
Epoch 51/60
132454/132454 [==============================] - 56s 419us/step - loss: 4.0002
*

and o'brien turned softh s th aere  tere t th   ar oh    he he he  o e the the a s tr th o eo  ther o e  he t   r a   th theth e  h te serj s the ee te e     a toh ro h oer t rher re   de   her  o  an sd s the t tn  o st no  ere oh ohe the he o   re he
s  t  e e e the oh mp e he the he thenou qthere o th teeo tr ee ere the ie  t d   
r te  e s  the th e ehe theo mh he e as mr he oe r t the thte  heh oe   he h there e th
----- temperature: 0.5
----- Generating with seed: ". i guess hands
and o'brien turned soft"
. i guess hands
and o'brien turned soft  h so e to aeteoeer ser ser  oeee te ro ar merrhesuri h tsre ts ne e che cnnee rhn hee h  se s h,ett th  t   ththe 
r c h no ere shrnhnou  m er o aoore a ere s a nrhe
he r tooa  a te, or thed t  o ee ho t ro, a me  ater i wrrhe   he  ther nore t  lo n 
 in tah  oo
aretht ro de srth othih oo s h 
t edor    e tecsreir a  tth t tare e aleat t thae s rrer otho ra o r s onnoedh mere hnc t en or ahe
----- temperature: 1.0
----- Generating with s

atyhs lrdes y s  htdonrhwu om h moy an is t orenhhsrthedin
i ntwa  dratmonr a win orucoy h thd amr ,ndo s oonahyud al 
adat  io u   a  mc muerama mr soh he sruniia  carn  t m thth,ttrhnndnouilrs t wland no
rr sharuin t e m tenle erheu tma eand, n nry m,
----- temperature: 1.2
----- Generating with seed: " for the merest trifle. there is a class"
 for the merest trifle. there is a classma
ynyon mih wiahaseranyt owwe toordml eo mi nrlm r osa
 iareydanct y srrm cmadr,  or  ,cy icaen ilei
ylt yan aohhala ayaee u la, ldna edduys,e mn
tndnsatheo  c dhr n
uma waynhrl tcmanr  namehawclsi todr md , yali ini uonswlldnee,  mnaii mhn mo  n s o r aes, uesca  hanena
n
osh hm,sn oceean  yom  wdameh e thhaned i  oe aldlosadasnhi ytou sare, mrotowal l nw hythaid shra wushm syoadl 
Epoch 58/60
132454/132454 [==============================] - 55s 417us/step - loss: 3.9999
****************************************************************************
----- Generating text after Epoch: 57
----- temperature: 